In [28]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
%matplotlib inline

nvs = pd.DataFrame()

def to_string(a):
    if isinstance(a, float):
        return str(int(a))
    elif isinstance(a, int):
        return str(a)
    else:
        return str(a)

os.chdir(r'C:\Users\AmanovRA\Текущее\НСК_Валере_2й_квартал\Новосибирск')
for path, folders, files in os.walk('.'):
    if not folders:
        for file in files:
            if file[-4:] == 'xlsx':
                source = f'Источник: {os.path.basename(path)}, Разделение: {file[:file.find(".")]}'
                temp_df = pd.read_excel(path + '\\' + file)
                temp_df['Источник'] = [source for i in range(temp_df.shape[0])]
                temp_df = temp_df.loc[:, ['Телефон', 'Источник']]
                temp_df = temp_df.drop(temp_df[temp_df['Телефон'].isnull()].index)
                temp_df['Телефон'] = temp_df['Телефон'].apply(to_string)
                nvs = nvs.append(temp_df, ignore_index=True)
        
pattern = r'\(\d{2}\)'
pattern2 = r'[а-яА-Яa-zA-Z]{1}'


mobil = nvs['Телефон'].to_list()
        

for i, j in enumerate(mobil):
    if type(j) == str:
        mobil[i] = ''.join([x if x not in ',;!:.\\/+' else ' ' for x in j])        
        mobil[i] = mobil[i].lstrip(' ')
        mobil[i] = mobil[i].rstrip(' ')
        if re.search(pattern2, mobil[i]):
            mobil[i] = re.sub(pattern2, '', mobil[i])
        mobil[i] = ''.join([x for x in mobil[i] if x not in '()-'])  
            

for i, j in enumerate(mobil):
    if type(j) == str:
        if len(j) == 7:
            mobil[i] = '7812' + mobil[i]
        elif len(j) == 11 and j[0] == '8':
            mobil[i] = '7' + j[1:]
        elif len(j) == 10:
            mobil[i] = '7' + j
            
            
nvs['Телефон'] = mobil   

# вручную немного говнономера отредактировал

In [44]:
nvs = pd.read_excel('temp.xlsx')

nvs = nvs.drop_duplicates(subset='Телефон').reset_index()

In [45]:
au_telefons = pd.read_excel('NSK_january_juny.xlsx')

dom, mobil, work = au_telefons['Телефон домашний'].tolist(), au_telefons['Телефон мобильный'].tolist(), au_telefons['Телефон рабочий'].tolist()

pattern = r'\(\d{2}\)'
pattern2 = r'[а-яА-Яa-zA-Z]{1}'

for key in (dom, mobil, work):
    for i, j in enumerate(key):
        if type(j) == str:
            key[i] = ''.join([x if x not in ',;!:.\\/+' else ' ' for x in j])        
            key[i] = key[i].lstrip(' ')
            key[i] = key[i].rstrip(' ')
            if re.search(pattern2, key[i]):
                key[i] = re.sub(pattern2, '', key[i])
            key[i] = ''.join([x for x in key[i] if x not in '()-'])  
            
for key in (dom, mobil, work):
    for i, j in enumerate(key):
        if type(j) == str:
            if len(j) == 7:
                key[i] = '7812' + key[i]
            elif len(j) == 11 and j[0] == '8':
                key[i] = '7' + j[1:]
            elif len(j) == 10:
                key[i] = '7' + j
                
def concate_lists(*args) -> list:   # соединим все номера в одну строку чтобы проще было искать
    temp_lst = []
    for i in range(len(args[0])):
        temp_lst.append(' '.join([str(x[i]) for x in args]))
    return temp_lst
            
lst_search = concate_lists(dom, mobil, work)  

lst_with_coincidenc_telefon = []
lst_with_coincidenc_source = []
for index, row in enumerate(lst_search):
    lst_with_coincidenc_telefon.append([])
    lst_with_coincidenc_source.append([])
    for i, j in enumerate(nvs['Телефон']):
        if str(j) in row:
            lst_with_coincidenc_telefon[index].append(str(j))
            lst_with_coincidenc_source[index].append(nvs['Источник'][i])
            break
            
au_telefons['NVS-телефон'], au_telefons['NVS-источник'] = lst_with_coincidenc_telefon, lst_with_coincidenc_source           
            
def join_list(lst):
    return ', '.join([str(i) for i in lst])

au_telefons['NVS-телефон'], au_telefons['NVS-источник'] = au_telefons['NVS-телефон'].apply(join_list), au_telefons['NVS-источник'].apply(join_list)

au_telefons.to_excel('NVS_Valere.xlsx')